In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV, RFE

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier

import random


In [ ]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [14]:
features = ["age", "highschool_diploma",  "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale", "Sephar_scale", "Unknown"]

In [ ]:

numerical_features = ["T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1",
               "state1", "PCL1",  "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", 
                 "PCL_Strict2", "cd_risc1", "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                 "religion1", "emotional_support1","instrumental_support1", "self_distraction1", "denial1", 
               "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2", "instrumental_support2", 
                 "self_distraction2", "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                 "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Ranked", "COMT_Hap1_recode", 
               "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5"]

categorical_features = ["age", "highschool_diploma",  "dyslexia", "ADHD","Ashken_scale", "Sephar_scale", "Unknown"]

# #df['bad_features'] = (df > df.mean())[bad_features].sum(axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])


for i in range(2):
    # how many features to take 
    amount_features = random.randint(3, 16)

    #which features to take
    random.shuffle(features)
    X = df[features[:amount_features]]

    print(features[:amount_features])

    ss = StandardScaler()
    X = ss.fit_transform(X)


    Y = df["PCL_Strict3"]

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
    X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)

    kfold = StratifiedKFold(n_splits=7, shuffle=True)
    cvscores = []
    y_train_2 = np.array(y_train_2)
    X_train_2 = np.array(X_train_2)

    for num_layers in [1, 0]:
        for first_layer in [75, 25]:
            for each_layer in [10]:
                for class_weight in [{1:1, 0:10}, {1:1, 0:2}]:
                    num_layers = num_layers
                    first_layer = first_layer
                    each_layer = 10
                    num_smote = 1
                    loops = 0
                    scores_f = []
                    scores_r = []
                    scores_p = []
                    print("num_layers", num_layers, "\nfirst_layer", first_layer, 
                              "\nclass_weight", class_weight, "\nnum_smote", num_smote)
                    for train, test in kfold.split(X_train_2, y_train_2):

                        sm = SMOTE(k_neighbors= 10)
                        X_train_res = X_train_2[train]
                        y_train_res = y_train_2[train]

                        X_train_res, y_train_res = sm.fit_sample(X_train_res, y_train_res.ravel())

                      # create model
                        n_cols = X_train_res.shape[1]
                        model = Sequential()
                        model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
                        model.add(Dropout(0.5))

                        for i in range(num_layers):
                            model.add(Dense(each_layer, activation='elu'))
                            model.add(Dropout(0.5))

                        model.add(Dense(1, activation='sigmoid'))

                        model.compile(optimizer='adam', 
                                      loss='binary_crossentropy')


                        # Fit the model
                        callbacks = [EarlyStopping(monitor='val_loss', patience=0)]
                        model.fit(X_train_res, y_train_res, epochs = 400, validation_split = .1, verbose=0, callbacks=callbacks, class_weight = class_weight)
                        # evaluate the model
                        y_pred =  model.predict(X_train_2[test])
                        y_train_pred =  model.predict(X_train_2[train])
                        y_pred = y_pred>0.5
                        s_f = f1_score(y_train_2[test], y_pred)
                        s_p = precision_score(y_train_2[test], y_pred)
                        s_r = recall_score(y_train_2[test], y_pred)
                        #print("\tscores f1", (s_f))
                        #print("\tscores p", (s_p))
                        #print("\tscores r", (s_r))
                        s = f1_score(y_pred,y_train_2[test])
                        #print(s)

                        scores_f.append(s_f)
                        scores_p.append(s_p)
                        scores_r.append(s_r)


                        #y_train_pred = (y_train_pred > 0.5)
                        #train_s_f = f1_score(y_train_2[train], y_train_pred)
                        #train_s_p = precision_score(y_train_2[train], y_train_pred)
                        #train_s_r = recall_score(y_train_2[train], y_train_pred)
                        #print("\tscores f1 train", (train_s_f))
                        #print("\tscores p train", (train_s_p))
                        #print("\tscores r train", (train_s_r))
                    print("mean precision scores", np.mean(scores_p))
                    print("mean recall scores", np.mean(scores_r))
                    print("mean f1 scores", np.mean(scores_f))
                    print ("\n_______________________\n")

['behavioral_disengagement1', 'PCL_Strict2', 'planning2', 'self_blame1', 'T1bias', 'trait2', 'Ashken_scale', 'PCL2', 'T1Acc1n', 'denial2', 'self_distraction2', 'lot1', 'state1', 'instrumental_support1']
num_layers 1 
first_layer 75 
class_weight {1: 1, 0: 10} 
num_smote 1
mean precision scores 0.19421768707482995
mean recall scores 0.2523809523809524
mean f1 scores 0.2159792588364017

_______________________

num_layers 1 
first_layer 75 
class_weight {1: 1, 0: 2} 
num_smote 1
mean precision scores 0.17104585055712876
mean recall scores 0.5619047619047619
mean f1 scores 0.26084941477226536

_______________________

num_layers 1 
first_layer 25 
class_weight {1: 1, 0: 10} 
num_smote 1
mean precision scores 0.13667509164403577
mean recall scores 0.43809523809523815
mean f1 scores 0.1934010319103487

_______________________

num_layers 1 
first_layer 25 
class_weight {1: 1, 0: 2} 
num_smote 1
mean precision scores 0.20484693877551022
mean recall scores 0.6047619047619047
mean f1 scores 0.

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predi

mean precision scores 0.0
mean recall scores 0.0
mean f1 scores 0.0

_______________________

num_layers 1 
first_layer 75 
class_weight {1: 1, 0: 2} 
num_smote 1


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predi

mean precision scores 0.0
mean recall scores 0.0
mean f1 scores 0.0

_______________________

num_layers 1 
first_layer 25 
class_weight {1: 1, 0: 10} 
num_smote 1


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
